In [ ]:
import ee
import geemap

ee.Initialize()

In [ ]:
values = ee.List.sequence(1, 10)
print(values.getInfo())

In [ ]:
count = values.reduce(ee.Reducer.count())
print(count.getInfo())

In [ ]:
values.size().getInfo()

In [ ]:
minimum = values.reduce(ee.Reducer.min())
print(minimum.getInfo())

In [ ]:
min_max = values.reduce(ee.Reducer.minMax())
min_max.getInfo()

In [ ]:
mean = values.reduce(ee.Reducer.mean())
print(mean.getInfo())

In [ ]:
std = values.reduce(ee.Reducer.stdDev())
print(std.getInfo())

# ImageCollection reducer

In [ ]:
Map = geemap.Map()

collection = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filterDate("2019-01-01", "2019-12-31")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
)

median = collection.reduce(ee.Reducer.median())

vis_params = {
    "bands": ["B5_median", "B4_median", "B3_median"],
    "gamma": 2,
}
Map.setCenter(-122.3578, 37.7726, 12)
Map.addLayer(median, vis_params, "Median Image")
Map

In [ ]:
collection.size().getInfo()

In [ ]:
image = ee.Image("LANDSAT/LC08/C01/T1/LC08_044034_20140318").select(["B4", "B3", "B2"])
maxValue = image.reduce(ee.Reducer.max())
Map.centerObject(image, 8)
Map.addLayer(image, {}, "Original image")
Map.addLayer(maxValue, {"max": 13000}, "Maximum value image")
Map

# Feature Collection Reducations

In [ ]:
Map = geemap.Map()
census = ee.FeatureCollection("TIGER/2010/Blocks")
benton = census.filter(
    ee.Filter.And(ee.Filter.eq("statefp10", "41"), ee.Filter.eq("countyfp10", "003"))
)
Map.setCenter(-123.27, 44.57, 13)
Map.addLayer(benton)
Map

In [ ]:
properties = ["pop10", "housing10"]
sums = benton.filter(ee.Filter.notNull(properties)).reduceColumns(
    **{"reducer": ee.Reducer.sum().repeat(2), "selectors": properties}
)
sums

In [ ]:
print(benton.aggregate_sum("pop10").getInfo())
print(benton.aggregate_sum("housing10").getInfo())

In [ ]:
benton.aggregate_stats("pop10").getInfo()

# Image discription

In [ ]:
Map = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])
image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(centroid).first()
vis = {"min": 0, "max": 3000, "bands": ["B5", "B4", "B3"]}

Map.centerObject(centroid, 8)
Map.addLayer(image, vis, "Landsat-8")
Map

In [ ]:
image.propertyNames()

In [ ]:
image.get("CLOUD_COVER")

In [ ]:
prop = image.propertyNames()
prop.getInfo()

In [ ]:
stats = geemap.image_stats(image)
stats.getInfo()

# Zonal statistics for two image layer

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
dem = ee.Image("USGS/3DEP/10m")
vis = {"min": 0, "max": 4000, "palette": "terrain"}
Map.addLayer(dem, vis, "DEM")
Map

In [ ]:
landcover = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select("landcover")
Map.addLayer(landcover, {}, "NLCD 2019")
Map.add_legend(title="NLCD Land Cover Classification", builtin_legend="NLCD")

In [ ]:
stats = geemap.image_stats_by_zone(dem, landcover, reducer="MEAN")
stats

In [ ]:
stats.to_csv("stats.csv", index=False)